In [29]:
import geopandas as gpd
import geojsonio as geoio
import pandas as pd
from collections import defaultdict

from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Read Data

In [2]:
vancouver_gdf = gpd.read_file('VancouverAllData.geojson')
osm_df = pd.read_json('amenities-vancouver.json', lines=True)
osm_gdf = gpd.GeoDataFrame(osm_df, geometry=gpd.points_from_xy(osm_df.lat, osm_df.lon))

# Data Prep

TO DO: igure out how to check whether a point is inside MultiPolygon

In [33]:
def isWithinMultipolygon(point, multipolygon):
    return False

Initialize amenities column with None values

In [36]:
vancouver_gdf['amenities'] = vancouver_gdf.apply(lambda x: None, axis=1)

For each Vancouver area, count the number of each amenity type that is in this area, e.g. someArea['amenities'] = {cafe: 2, bank: 4,...}<br>
TO DO: this needs to be optimized since two for loops take way too long

In [ ]:
for i in range(len(vancouver_gdf)):
    amenities = defaultdict(int)
    for j in range(len(osm_gdf)):
        if isWithinMultipolygon(osm_gdf.iloc[[j]], vancouver_gdf.iloc[[i]]):
            amenities[osm_gdf.iloc[[0]]['amenity']] += 1
    vancouver_gdf.at[i, 'amenities'] = amenities

# Machine Learning

Our x is the number of amenities of some type in the area (e.g. 2 cafes, 4 banks, etc.). Our y is the population density in this area.


In [ ]:
X = vancouver_gdf['amenities'].values
y = vancouver_gdf['pop'].values # divide by the area of the multipolygon 
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
# train model
model = make_pipeline(
    GaussianNB()
)
model.fit(X_train, y_train)
# validate model
print(model.score(X_valid, y_valid))

# Heatmap

Red: overpopulated
Yellow: OK
Green: underpopulated

# Analysis